# Mod 21 - SQL - Exercício 2

## Imports e Funções

In [2]:
import psycopg2 as pg2
import pandas as pd

In [3]:
def connect_to_postgress(dbname:str) -> pg2.connect:
    conn = None
    try:
        conn = pg2.connect(
            host='localhost',
            port=5432,
            dbname=dbname,
            user='postgres',
            password='1234'
        )
    except Exception as e:
        print(e)
    
    return conn

def get_result_query_df(conn: pg2.connect, query: str) -> pd.DataFrame:
    data = None
    try:
        cur = conn.cursor()

        cur.execute(query)
        conn.commit()

        data = cur.fetchall()
        colnames = [desc.name for desc in cur.description]

        cur.close()
        conn.close()
        
        data = pd.DataFrame(data, columns=colnames)
        
    except Exception as e:
        print(e)
    
    return data

## Tarefa 1:

Calcule a Média por nome e sobrenome do ator das seguintes variáveis:

- rental_duration
- rental_rate
- length
- replacement_coast

In [4]:
query_str = '''
    SELECT * 
    FROM film AS t1
    LEFT JOIN film_actor AS t2
    ON t1.film_id = t2.film_id
    LEFT JOIN actor AS t3
    ON t2.actor_id = t3.actor_id
'''

df = get_result_query_df(conn=connect_to_postgress('dvdrental'), query=query_str)
df.head()

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext,actor_id,film_id,last_update,actor_id,first_name,last_name,last_update
0,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur...,1.0,1.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
1,23,Anaconda Confessions,A Lacklusture Display of a Dentist And a Denti...,2006,1,3,0.99,92,9.99,R,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]",'anaconda':1 'australia':18 'confess':2 'denti...,1.0,23.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
2,25,Angels Life,A Thoughtful Display of a Woman And a Astronau...,2006,1,3,2.99,74,15.99,G,2013-05-26 14:50:58.951,[Trailers],'angel':1 'astronaut':11 'battl':14 'berlin':1...,1.0,25.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
3,106,Bulworth Commandments,A Amazing Display of a Mad Cow And a Pioneer w...,2006,1,4,2.99,61,14.99,G,2013-05-26 14:50:58.951,[Trailers],'amaz':4 'bulworth':1 'command':2 'cow':9 'dis...,1.0,106.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620
4,140,Cheaper Clyde,A Emotional Character Study of a Pioneer And a...,2006,1,6,0.99,87,23.99,G,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Behind the Scenes]",'ancient':19 'charact':5 'cheaper':1 'clyde':2...,1.0,140.0,2006-02-15 10:05:03,1.0,Penelope,Guiness,2013-05-26 14:47:57.620


In [5]:
df['rental_rate'] = df['rental_rate'].astype(float)
df['replacement_cost'] = df['replacement_cost'].astype(float)

In [7]:
response = df.groupby(['first_name', 'last_name'])[['rental_duration', 'rental_rate', 'length', 'replacement_cost']].mean()
response

rental_duration  rental_rate      length  \
first_name last_name                                             
Adam       Grant             5.277778     2.767778  122.388889   
           Hopper            4.863636     2.808182  112.272727   
Al         Garland           5.115385     2.759231  106.384615   
Alan       Dreyfuss          5.037037     2.841852  111.296296   
Albert     Johansson         5.060606     3.293030  126.636364   
...                               ...          ...         ...   
Will       Wilson            5.064516     3.054516  115.967742   
William    Hackman           4.777778     3.138148  129.074074   
Woody      Hoffman           4.580645     2.990000  120.838710   
           Jolie             4.354839     2.602903  114.935484   
Zero       Cage              4.840000     2.830000  113.240000   

                      replacement_cost  
first_name last_name                    
Adam       Grant             22.934444  
           Hopper            19.490000  
Al         Garland           19.874615  
Alan       Dreyfuss          19.360370  
Albert     Johansson         22.474848  
...                                ...  
Will       Wilson            19.957742  
William    Hackman           20.064074  
Woody      Hoffman           19.699677  
           Jolie             19.409355  
Zero       Cage              19.910000  

[199 rows x 4 columns]

## Tarefa 2:

Calcule soma de amount (preço total do aluguel) por nome, sobrenome e email do cliente (customer)

In [8]:
query_str = '''
    SELECT * 
    FROM payment AS t1
    LEFT JOIN customer AS t2
    ON t1.customer_id = t2.customer_id
'''

df = get_result_query_df(conn=connect_to_postgress('dvdrental'), query=query_str)
df.head()

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577,341,1,Peter,Menard,peter.menard@sakilacustomer.org,346,True,2006-02-14,2013-05-26 14:49:45.738,1


In [10]:
df['amount'] = df['amount'].astype(float)

In [12]:
response = df.groupby(['email', 'first_name', 'last_name'])[['amount']].sum()
response

,,,amount
email,first_name,last_name,
aaron.selby@sakilacustomer.org,Aaron,Selby,88.80
adam.gooch@sakilacustomer.org,Adam,Gooch,97.80
adrian.clary@sakilacustomer.org,Adrian,Clary,68.82
agnes.bishop@sakilacustomer.org,Agnes,Bishop,86.79
alan.kahn@sakilacustomer.org,Alan,Kahn,119.75
...,...,...,...
willie.markham@sakilacustomer.org,Willie,Markham,85.78
wilma.richards@sakilacustomer.org,Wilma,Richards,91.80
yolanda.weaver@sakilacustomer.org,Yolanda,Weaver,102.75
